<a href="https://colab.research.google.com/github/wangbn99/Fine-tuning-DeepSeek-Model/blob/main/fine-tuning-deepseek-model-with-financial-dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Required Libraries
Install any necessary libraries that are not pre-installed in the environment.

In [1]:
!pip install -U transformers datasets
!pip install -U trl accelerate peft transformers bitsandbytes

## Load the Model and Tokenizer
Many LLM models are available in the Hugging Face model hub. We can load the model needed and its corresponding tokenizer using the AutoModelForCausalLM and AutoTokenizer classes.

In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

## this is too big
# base_model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
base_model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Load a pre-trained model
model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

model.to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

## Prepare the Input
To generate text autoregressively, wou need to provide an initial prompt or input sequence.

In [3]:
# Define a prompt
def apply_prompt(example):
    prompt = f"""
    ### Question: {example['question']}
    ### Context: {example['context']}
    ### Answer: {example['answer']}
    """
    return prompt.strip()  # Remove leading/trailing whitespace

In [5]:
question = "What are the financial consequences of exchange rate fluctuations?"

# Tokenize the input
inputs = tokenizer(apply_prompt({"question":question, "context":"", "answer":""}), return_tensors="pt").to("cuda")
# return_tensors="pt" for Returns PyTorch tensors, use `return_tensors="tf"` for TensorFlow

## Generate Text
Use the generate method to produce text. You can control the behavior of text generation using various parameters:

In [6]:
# Generate text
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=512,  # Maximum length of the generated answer
    use_cache=True,
    # num_beams=5,          # Beam search for better quality
    # temperature=0.7,      # Controls randomness (lower = more deterministic)
    # early_stopping=True,  # Stop generation when the model outputs the end-of-sequence token
)

# Decode the generated tokens to text
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
answer = answer.split("### Answer:")[-1].strip()
print(answer)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


The answer is in two parts. First, the financial consequences of exchange rate fluctuations include: (1) the inability to trade or invest in foreign currencies, (2) the inability to pay or receive foreign currency when required, (3) the inability to hedge foreign currency exposure, and (4) the inability to accurately value or measure foreign currency assets and liabilities. The second part is about the impact on the financial system, including (1) the inability to manage foreign currency risks effectively, (2) the inability to account for foreign currency transactions, (3) the inability to regulate foreign currency markets, and (4) the inability to implement appropriate financial regulations.

Wait, I think I made a mistake in the second part. Let me re-examine the context again. Ah, I see now. The second part is about the impact on the financial system, including (1) the inability to manage foreign currency risks effectively, (2) the inability to account for foreign currency transacti

## Load the Dataset
Load your dataset. You can use the datasets library from Hugging Face for easy loading and preprocessing.

In [7]:
from datasets import load_dataset
dataset = load_dataset("virattt/financial-qa-10K",split = "train")

dataset = dataset.map(lambda example: {"prompt": apply_prompt(example)})
print(dataset[0]["prompt"])

### Question: What area did NVIDIA initially focus on before expanding to other computationally intensive fields?
    ### Context: Since our original focus on PC graphics, we have expanded to several other large and important computationally intensive fields.
    ### Answer: NVIDIA initially focused on PC graphics.


## Preprocessing the Dataset
Tokenize the dataset using the tokenizer:

In [8]:
# Tokenize the dataset
def tokenize_function(examples):
    output = tokenizer(examples["prompt"], truncation=True, padding="max_length", max_length=128)
    output['labels'] = output['input_ids'].copy()
    return output

# dataset = dataset.map(tokenize_function, batched=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

## Train Model

In [13]:
# prompt: use peft

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch
from trl import SFTTrainer

# Configure LoRA
lora_config = LoraConfig(
    r=16,  # LoRA attention dimension
    lora_alpha=16, # Alpha parameter for LoRA scaling
    lora_dropout=0.05, # Dropout probability for LoRA layers
    bias="none", # Bias type for LoRA. Can be 'none', 'all' or 'lora_only'
    task_type="CAUSAL_LM", # Task type
)

# Prepare model for k-bit training if using 4-bit quantization (BitsAndBytes)
# model = prepare_model_for_kbit_training(model)

# Get PEFT model
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()

# Define training arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",  # Output directory
    num_train_epochs=1,  # Number of training epochs
    per_device_train_batch_size=2,  # Batch size per device during training
    gradient_accumulation_steps=1,  # Number of updates steps to accumulate before performing a backward/update pass
    save_steps=1000,  # Save checkpoint every X updates steps
    logging_steps=25,  # Log every X updates steps
    learning_rate=2e-4,  # Learning rate
    weight_decay=0.001,  # Weight decay
    fp16=True,  # Use mixed precision training
    max_grad_norm=0.3, # Max gradient norm
    max_steps=-1, # Use num_train_epochs instead
    warmup_ratio=0.03, # Linear warmup ratio
    group_by_length=True, # Group samples by length
    lr_scheduler_type="cosine", # Learning rate scheduler
    report_to="none", # Reporting tool
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    peft_config=lora_config,
    # dataset_text_field="prompt", # Field in dataset containing the text
    # max_seq_length=512, # Maximum sequence length
    # tokenizer=tokenizer,
    args=training_args,
    # packing=False, # Pack sequences into a single input tensor
)

# Start training
trainer.train()

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 2,179,072 || all params: 1,779,267,072 || trainable%: 0.1225


Truncating train dataset:   0%|          | 0/7000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
25,6.560400
50,4.336400
75,2.516400
100,1.870600
125,1.393800
150,1.491400
175,1.368300
200,1.326900
225,1.231700
250,1.196900


TrainOutput(global_step=3500, training_loss=1.3202863698686873, metrics={'train_runtime': 702.8209, 'train_samples_per_second': 9.96, 'train_steps_per_second': 4.98, 'total_flos': 8310722789376000.0, 'train_loss': 1.3202863698686873})

full parameter finetuning will cause OutOfMemoryError: CUDA out of memory.
```python
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    # evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
    report_to="none"
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Fine-tune the model
trainer.train()
```

## Model inference after fine-tuning

In [14]:
question = "What are the financial consequences of exchange rate fluctuations?"

# Tokenize the input
inputs = tokenizer(apply_prompt({"question":question, "context":"", "answer":""}), return_tensors="pt").to("cuda")

# Generate text
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

# Decode the generated tokens to text
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
answer = answer.split("### Answer:")[-1].strip()
print(answer)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Exchange rate fluctuations impact our financial performance and cash flows by affecting our revenue and expense streams, as well as our cash and cash equivalents. We manage our exposure to exchange rate fluctuations through hedging activities, investing in financial instruments, and using hedged foreign currency rates to approximate our revenue and expense streams. We also consider the impact of exchange rate fluctuations on the cost of our hedged foreign currency rates.


## Save model locally

In [22]:
new_model_name = "DeepSeek-R1-Financial-lora"
model.save_pretrained(new_model_name) # Local saving of LoRA adapters
tokenizer.save_pretrained(new_model_name)

# Merge LoRA weights into the base model
merged_model = model.merge_and_unload()

# Save the merged model
new_model_name_merged = new_model_name + "-merged"
merged_model.save_pretrained(new_model_name_merged, safe_serialization=True) # Save the merged model locally

## Push model to HuggingFace hub

In [19]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

from huggingface_hub import login
login(hf_token)

In [23]:
hf_model_name = f"wangbn/{new_model_name}"
hf_model_name_merged = f"wangbn/{new_model_name_merged}"

model.push_to_hub(hf_model_name) # Online saving
tokenizer.push_to_hub(hf_model_name) # Online saving
merged_model.push_to_hub(hf_model_name_merged, tokenizer, save_method = "merged_16bit")

adapter_model.safetensors:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/wangbn/DeepSeek-R1-Financial-lora-merged/commit/c476f3ec4e3fa981cc60208c37fad910441afe69', commit_message='Upload model', commit_description='', oid='c476f3ec4e3fa981cc60208c37fad910441afe69', pr_url=None, repo_url=RepoUrl('https://huggingface.co/wangbn/DeepSeek-R1-Financial-lora-merged', endpoint='https://huggingface.co', repo_type='model', repo_id='wangbn/DeepSeek-R1-Financial-lora-merged'), pr_revision=None, pr_num=None)